In [2]:
from collections import Counter
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB


df = pd.read_csv('./documentos/documentos.csv')
df

,Documento,Palabras,Clase
0,1,I loved the movie,+
1,2,I hated the movie,-
2,3,A great movie. Good movie.,+
3,4,Poor acting,-
4,5,Great acting. A good movie.,+
5,6,Good movie but poor acting,+


In [3]:
#Funciones
def limpiarString(cadena, caracteresAremplazar, nuevoValor):
    
    for elem in caracteresAremplazar:
       
        if elem in cadena:
            
            cadena = cadena.replace(elem, nuevoValor)
    
    return  cadena

def sacarPalabrasDiff(lista):
    cnt = Counter()
    for x in lista:
        cnt[x]+=1
        
    return cnt  

def crearListaDePalabrasDiff(diccionario):
    lista = []
    for i in diccionario.keys():
        lista.append(i)
    
    return lista

def probabilidadDeLasPalabras(listaPalabrasDiff, listaPalabrasClase, totalPalabrasDelaClase, palabrasTotales):
    diccionario = {}
    for x in listaPalabrasDiff:
        diccionario["{0}".format(x)] = ((listaPalabrasClase.count(x)+1)/(totalPalabrasDelaClase+palabrasTotales))
    
    return diccionario

def proporcion(probabilidadDelaClase, documento, diccProbabilidadClase, totalPalabrasDelaClase, palabrasTotales):
    calculo = probabilidadDelaClase
    for x in documento:
        if x in diccProbabilidadClase:
            calculo *= diccProbabilidadClase[x]
        else:
            calculo *= ((0+1)/(totalPalabrasDelaClase+palabrasTotales))
    return calculo

In [4]:
#Crear una lista con las positivas, negativas y con todas las frases
x = df[['Palabras', 'Clase']]
palabras = x.values.tolist()
listaPositivas = []
listaNegativas = []
documentos = []
#print(palabras)
for x in range(len(palabras)):
    documentos.append(palabras[x][0])
    if palabras[x][1] == '+':
        #data.append(palabras[x])
        listaPositivas.append(palabras[x][0])
    elif palabras[x][1] == '-':
        #data.append(palabras[x])
        listaNegativas.append(palabras[x][0])

print(palabras)
#print(listaPositivas)
#print(listaNegativas)
#print(documentos)

[['I loved the movie', '+'], ['I hated the movie', '-'], ['A great movie. Good movie.', '+'], ['Poor acting', '-'], ['Great acting. A good movie.', '+'], ['Good movie but poor acting', '+']]


In [5]:
cadena = ""
cadenaPositivas = ""
cadenaNegativas = ""
caracteres = ['.', ',', '\n']

#Vocabulario total
for x in documentos:
    cadena += x+" "

cadena = limpiarString(cadena, caracteres, "")
cadena = cadena.upper()
listaPalabras = cadena.split()
#print(listaPalabras)

diccVocabulario = sacarPalabrasDiff(listaPalabras)
#print(diccVocabulario)
listaVocabulario = crearListaDePalabrasDiff(diccVocabulario)
vocabulario = len(listaVocabulario)

#Clase positiva 
for x in listaPositivas:
    cadenaPositivas += x+" "
    
positivas = limpiarString(cadenaPositivas, caracteres , "") 
positivas = positivas.upper()
positivas = positivas.split()

#Creo un diccionario con las palabras y las veces que se repiten
cntpositivas = sacarPalabrasDiff(positivas)

#Creo una lista con las keys del diccionario
diffpositivas = crearListaDePalabrasDiff(cntpositivas)

totalpositivas = len(positivas)

#Clase negativa
for x in listaNegativas:
    cadenaNegativas+=x+" "

negativas = limpiarString(cadenaNegativas, caracteres, "")
negativas = negativas.upper()
negativas = negativas.split()

cntnegativas = sacarPalabrasDiff(negativas)

diffnegativas = crearListaDePalabrasDiff(cntnegativas)

totalnegativas = len(negativas)

Ppositiva = len(listaPositivas)/len(documentos)
Pnegativa = len(listaNegativas)/len(documentos)

print(f"Vocabulario: {vocabulario}")
print(f"Total palabras positivas: {totalpositivas}")
print(f"Total palabras negativas: {totalnegativas}")
print(f"P +: {Ppositiva}")
print(f"P -: {Pnegativa}")

Vocabulario: 11
Total palabras positivas: 19
Total palabras negativas: 6
P +: 0.6666666666666666
P -: 0.3333333333333333


In [6]:
dpositivas = {}
dnegativas = {}

dpositivas = probabilidadDeLasPalabras(diffpositivas, positivas, totalpositivas, vocabulario)

dnegativas = probabilidadDeLasPalabras(diffnegativas, negativas, totalnegativas, vocabulario)

print("DOCUMENTO NUEVO")

documentoNuevo = "good movie but poor acting"

documentoNuevo = limpiarString(documentoNuevo, caracteres, "")
documentoNuevo = documentoNuevo.upper()
documentoNuevo = documentoNuevo.split()

proporcionClasePositivo = proporcion(Ppositiva,documentoNuevo,dpositivas,totalpositivas,vocabulario)

proporcionClaseNegativa = proporcion(Pnegativa,documentoNuevo,dnegativas,totalnegativas,vocabulario)
            

print(f"Proporcion del que el documento sea de la clase positiva {proporcionClasePositivo}")
print(f"Proporcion del que el documento sea de la clase negativo {proporcionClaseNegativa}")        

if proporcionClasePositivo > proporcionClaseNegativa:
    print("Este documento tiene mas proporcion de la clase positivo")
elif proporcionClasePositivo < proporcionClaseNegativa:
    print("Este documento tiene mas proporcion de la clase negativa")
elif proporcionClaseNegativa == proporcionClasePositivo:
    print("Este documento tiene la misma proporcion en las dos clases")
    

DOCUMENTO NUEVO
Proporcion del que el documento sea de la clase positiva 7.901234567901235e-06
Proporcion del que el documento sea de la clase negativo 1.8781234072633135e-06
Este documento tiene mas proporcion de la clase positivo


In [7]:
data = pd.DataFrame(palabras, columns = ['message', 'class'])
data

,message,class
0,I loved the movie,+
1,I hated the movie,-
2,A great movie. Good movie.,+
3,Poor acting,-
4,Great acting. A good movie.,+
5,Good movie but poor acting,+


In [12]:
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(data['message'].values)
#print(counts)
classifier = MultinomialNB()
targets = data['class'].values
classifier.fit(counts, targets)

MultinomialNB()

In [11]:
examples = ["Good movie"]
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions

array(['+'], dtype='<U1')